# A Importância da Aleatoridade

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [2]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy = 'stratified')
dummy_stratified.fit(treino_x, treino_y)
#previsoes = dummy.predict(teste_x)

acuracia = dummy_stratified.score(teste_x, teste_y) * 100
print("A acurácia do dummy foi %.2f%%" % acuracia)

A acurácia do dummy foi 50.96%


In [3]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [4]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia foi 76.84%


In [5]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 3)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('Accuracy = [%.2f, %.2f]' % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy = [74.99, 76.57]


In [18]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('Accuracy = [%.2f, %.2f]' % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy = [74.24, 77.32]


In [19]:
SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 5)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()

print('Accuracy = [%.2f, %.2f]' % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

Accuracy = [75.21, 76.35]


Por padrão o valor usado para cv é 5, a literatura fala de valores entre 5 e 10, e o cross_validate usa por padrão o cv = 5

# Aleatoridade no cross validate

In [14]:
def imprime_resultados(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()

    print('Accuracy = %.2f' % (media * 100))
    print('Accuracy = [%.2f, %.2f]' % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao)*100))

In [27]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv)
imprime_resultados(results)

Accuracy = 75.78
Accuracy = [74.37, 77.19]


In [28]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv)
imprime_resultados(results)

Accuracy = 75.76
Accuracy = [73.26, 78.26]


# Simular situação horrível de azar
Pode ser 'azar' como pode ser proporção de exemplos desbalanceado entre as classes

In [6]:
dados_azar = dados.sort_values('vendido', ascending = True)

x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


In [31]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultados(results)

Accuracy = 57.84
Accuracy = [34.29, 81.39]


In [32]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultados(results)

Accuracy = 75.78
Accuracy = [72.30, 79.26]


In [36]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits = 10, shuffle = True)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv)
imprime_resultados(results)

Accuracy = 75.78
Accuracy = [73.55, 78.01]


# Gerando dados aleatórios

In [7]:
np.random.seed(SEED)

dados['modelo_aleatorio'] = dados['idade_do_modelo'] + np.random.randint(-2, 3, size = 10000)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,2
4,117384.68,1,4,12770.11290,2


In [8]:
dados.modelo_aleatorio.unique()

array([17, 20, 12,  2, 10,  9, 18,  4, 16,  6,  5, 21, 15, 13, 11, 14,  8,
        3, 19,  7, 22,  0,  1, -1], dtype=int64)

In [9]:
dados['modelo'] = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1
dados.modelo.min()

1

In [56]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

In [15]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo)
imprime_resultados(results)

Accuracy = 75.76
Accuracy = [72.91, 78.60]


# Cross validation com StandardScaler

In [12]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_esc = scaler.transform(treino_x)
teste_x_esc = scaler.transform(teste_x)

SEED = 301
np.random.seed(SEED)

modelo = SVC()
modelo.fit(treino_x_esc, treino_y)
previsoes = modelo.predict(teste_x_esc)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)


A acurácia foi 77.48%


In [16]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)

modelo = SVC()
results = cross_validate(modelo, x_azar, y_azar, cv = cv, groups = dados.modelo)
imprime_resultados(results)

Accuracy = 77.24
Accuracy = [73.71, 80.77]


In [17]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_esc = scaler.transform(x_azar)
x_azar_esc

array([[ 0.3636103 , -0.39591706,  0.24651152],
       [ 0.79214444, -0.18257669,  0.03346607],
       [ 0.72310419,  1.09746554,  1.66435195],
       ...,
       [ 0.17822399, -0.60925743, -0.73405662],
       [ 0.2248779 ,  0.45744443, -0.39741686],
       [-1.34266351,  0.88412517,  1.52099053]])

In [18]:
SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)

modelo = SVC()
results = cross_validate(modelo, x_azar_esc, y_azar, cv = cv, groups = dados.modelo)
imprime_resultados(results)

Accuracy = 76.65
Accuracy = [73.60, 79.69]


In [20]:
from sklearn.pipeline import Pipeline
SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)

results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo)
imprime_resultados(results)

Accuracy = 76.66
Accuracy = [73.65, 79.66]
